In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
DATA_DIR = '/content/drive/Shareddrives/Intelligent System - Assignment /Data/'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import string
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
data = pd.read_csv(DATA_DIR+'evnexpress/items3.csv')
data.head()

,main_class,sub_class,title,body,tag
0,news,news,Hanoi sends rhino horn trader to jail for 14 y...,"Toan, 36, made false customs declarations to a...","Vietnam,wildlife,rhino horn,conservation,envir..."
1,news,news,Vietnam Expressway Corporation officials jaile...,"A Hanoi court handed Le Quang Hao, 45, and Ngu...","Vietnam,Vietnam officials,Da Nang - Quang Ngai..."
2,news,news,HCMC plans third Covid shots starting Friday,In a draft plan regarding Covid-19 vaccination...,"Vietnam,Vietnamese,Covid-19,coronavirus,HCMC,H..."
3,news,news,HCMC metro line takes possession of six train ...,"The cars, belonging to the 8th and 9th trains ...","Vietnam,Vietnamese,HCMC,Ho Chi Minh,Saigon,tra..."
4,news,news,"Understaffed, underpaid: tough working conditi...",Thach Hung was preparing medicines for new Cov...,"Vietnam,HCMC,HCMC Covid-19,HCMC outbreak,HCMC ..."


In [ ]:
titleField = 'title'
bodyField = 'body'
labelField = 'main_class'

In [ ]:
# Remove video
data = data[data[labelField]!='video']
data[labelField].replace({'world': 'news'}, inplace=True)
categories = data[labelField].unique()
categories

array(['news', 'business', 'travel', 'life', 'sports'], dtype=object)

In [ ]:
data.shape

(5292, 5)

In [ ]:
def preprocessing(inp):
  text = str(inp)
  tokens = [re.sub(r'[^a-z|A-Z]', ' ', word.strip().lower()) for word in text.split()]
  

  return " ".join(tokens)

In [ ]:
data[bodyField][0]

"Toan, 36, made false customs declarations to avoid checks and brought 55 pieces of rhino horns weighing 126.5 kilograms, from the United Arab Emirates (UAE) into Vietnam in July 2019. Hanoi customs officers and police detected the violation and investigated it further.Under Vietnam's Penal Code, the maximum jail term for holding, trading, and trafficking rhino horns is 15 years. The Education for Nature Vietnam (ENV), a non-governmental organization, said this punishment is the toughest one to date in Vietnam for crimes relating to rhino horns and wildlife trade.Bui Thi Ha, ENV vice director, said amid the negative opinions about Vietnam’s handling of wildlife crimes, this heavy penalty showed that Vietnam’s competent authorities have begun to mete out serious punishment to deter people from wildlife crimes.A problem with this case is that the authorities could not detect the people who hired Toan to commit the crime, who are the main culprits, Ha said, as cited by newspaper.According

In [ ]:
preprocessing(preprocessing(data.loc[0, bodyField]))

'toan made false customs declarations to avoid checks and brought pieces of rhino horns weighing kilograms from the united arab emirates uae into vietnam in july hanoi customs officers and police detected the violation and investigated it further under vietnam s penal code the maximum jail term for holding trading and trafficking rhino horns is years the education for nature vietnam env a non governmental organization said this punishment is the toughest one to date in vietnam for crimes relating to rhino horns and wildlife trade bui thi ha env vice director said amid the negative opinions about vietnam s handling of wildlife crimes this heavy penalty showed that vietnam s competent authorities have begun to mete out serious punishment to deter people from wildlife crimes a problem with this case is that the authorities could not detect the people who hired toan to commit the crime who are the main culprits ha said as cited by newspaper according to the env since vietnam has recorded c

In [ ]:
data['text'] = data[titleField] + ' ' + data[bodyField]
data['text'] = data['text'].apply(preprocessing).apply(preprocessing)
data['text']

0       hanoi sends rhino horn trader to jail for year...
1       vietnam expressway corporation officials jaile...
2       hcmc plans third covid shots starting friday i...
3       hcmc metro line takes possession of six train ...
4       understaffed underpaid tough working condition...
                              ...                        
5293    step change in energy investment needed to mee...
5294    india votes in penultimate phase of seven roun...
5295    myanmar plane in emergency touchdown as landin...
5296    trump to china trade deal now or it will be fa...
5297    a new climate strike opting for no children as...
Name: text, Length: 5292, dtype: object

In [ ]:
X = data['text'].values

In [ ]:
text = data['text']

In [ ]:
LE = LabelEncoder()
y = pd.get_dummies(data[labelField])
display(y)
y = y.values

,business,life,news,sports,travel
0,0,0,1,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,1,0,0
...,...,...,...,...,...
5293,0,0,1,0,0
5294,0,0,1,0,0
5295,0,0,1,0,0
5296,0,0,1,0,0


## 1. Doc2Vec + CNN

In [ ]:
max_epochs = 5
vec_size = 50

In [ ]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text.values.tolist())]

In [ ]:
model = Doc2Vec(vector_size=vec_size,
                min_count=5,
                dm=1,
                window=3,
                epochs=10)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_crawl.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
Model Saved


In [ ]:
# model = Doc2Vec.load("/content/drive/Shareddrives/Intelligent System - Assignment /Models/d2v_crawl.model")
model = Doc2Vec.load("d2v_crawl.model")
v1 = model.infer_vector(text[0])
v1

array([-0.30177644, -0.07809409, -0.0744546 ,  0.58461004,  0.13925672,
       -0.02016563, -0.10985368,  0.32327992, -0.04322156, -0.2798818 ,
       -0.49303088,  0.10368802,  0.24690503,  0.01952123, -0.0715822 ,
        0.29313242, -0.08040892,  0.10607806,  0.15143262,  0.09798054,
        0.13493457,  0.21435802, -0.09154264, -0.27300885,  0.01644099,
        0.06729474, -0.19933625, -0.15844473,  0.22056618, -0.19614564,
       -0.21989776, -0.38721704, -0.0940379 , -0.07905596,  0.08741602,
       -0.15621582,  0.24111564,  0.08431815,  0.21847333,  0.15485606,
       -0.36976254,  0.18131316, -0.12260368,  0.33166683, -0.15857807,
        0.26938653,  0.0951661 , -0.07269589,  0.12942946,  0.43719873],
      dtype=float32)

In [ ]:
input_matrix = np.zeros((text.shape[0], vec_size))

In [ ]:
for i, d in enumerate(text):
  input_matrix[i] = model.infer_vector(str(d))

In [ ]:
X = input_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=69)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4233, 50), (1059, 50), (4233, 5), (1059, 5))

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, kernel_size=3, activation='relu', input_shape=(50, 1)),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 48, 16)            64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 16)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 32)            1568      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 11, 32)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 9, 64)             6208      
                                                                 
 flatten (Flatten)           (None, 576)               0

In [ ]:
model.fit(X_train[:, :, np.newaxis], y_train, epochs=100, batch_size=256)

Epoch 1/100
17/17 [==============================] - 13s 14ms/step - loss: 1.4977 - accuracy: 0.3570
Epoch 2/100
17/17 [==============================] - 0s 8ms/step - loss: 1.3164 - accuracy: 0.4713
Epoch 3/100
17/17 [==============================] - 0s 8ms/step - loss: 1.1759 - accuracy: 0.5370
Epoch 4/100
17/17 [==============================] - 0s 8ms/step - loss: 1.1038 - accuracy: 0.5807
Epoch 5/100
17/17 [==============================] - 0s 8ms/step - loss: 1.0429 - accuracy: 0.6057
Epoch 6/100
17/17 [==============================] - 0s 8ms/step - loss: 0.9932 - accuracy: 0.6286
Epoch 7/100
17/17 [==============================] - 0s 9ms/step - loss: 0.9663 - accuracy: 0.6419
Epoch 8/100
17/17 [==============================] - 0s 8ms/step - loss: 0.9441 - accuracy: 0.6497
Epoch 9/100
17/17 [==============================] - 0s 8ms/step - loss: 0.9153 - accuracy: 0.6539
Epoch 10/100
17/17 [==============================] - 0s 9ms/step - loss: 0.9051 - accuracy: 0.6589
Epoch 1

In [ ]:
y_pred = model.predict(X_test[:, :, np.newaxis])
y_pred

array([[5.7111602e-02, 3.1559631e-02, 9.1130793e-01, 4.7410680e-07,
        2.0429348e-05],
       [1.6247229e-01, 1.0112711e-04, 8.3734727e-01, 7.6130171e-05,
        3.1026482e-06],
       [6.9745685e-09, 2.7121017e-05, 9.7135317e-01, 6.5839140e-11,
        2.8619731e-02],
       ...,
       [2.0991520e-03, 2.4912558e-02, 1.4669283e-01, 8.2628608e-01,
        9.4878133e-06],
       [2.2356209e-01, 1.2226545e-03, 7.7509075e-01, 1.2490434e-05,
        1.1206956e-04],
       [4.5130655e-04, 2.5768399e-05, 9.9617237e-01, 8.5736495e-08,
        3.3505170e-03]], dtype=float32)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.69      0.75      0.72       293
           1       0.51      0.40      0.44       106
           2       0.73      0.74      0.73       370
           3       0.77      0.91      0.83       190
           4       0.61      0.35      0.45       100

    accuracy                           0.70      1059
   macro avg       0.66      0.63      0.64      1059
weighted avg       0.69      0.70      0.69      1059



In [ ]:
model.evaluate(X_test[:, :, np.newaxis], y_test)

34/34 [==============================] - 0s 5ms/step - loss: 1.5668 - accuracy: 0.7016


[1.5668377876281738, 0.7016052603721619]

In [ ]:
# Analysis on wrong predictions
wrong_indices = np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1)
wrong_test = y_test[wrong_indices]
wrong_pred = y_pred[wrong_indices]
for i in range(min(wrong_test.shape[0], 100)):
  print('Wrong sample {}'.format(i))
  print(wrong_test[i])
  print(wrong_pred[i])

Wrong sample 0
[0 0 0 0 1]
[5.7111602e-02 3.1559631e-02 9.1130793e-01 4.7410680e-07 2.0429348e-05]
Wrong sample 1
[0 0 0 0 1]
[9.8451310e-01 8.1156706e-04 1.4673953e-02 5.4907713e-07 7.7562368e-07]
Wrong sample 2
[0 0 1 0 0]
[0.00175226 0.40748727 0.06155882 0.00574872 0.52345294]
Wrong sample 3
[1 0 0 0 0]
[2.4892426e-08 2.3456528e-04 9.9974924e-01 1.9762097e-06 1.4225128e-05]
Wrong sample 4
[0 1 0 0 0]
[8.2724440e-01 1.5815102e-02 1.5693897e-01 1.0573443e-06 5.4141032e-07]
Wrong sample 5
[0 0 0 1 0]
[0.02169511 0.02624738 0.8188473  0.13131133 0.00189887]
Wrong sample 6
[0 0 1 0 0]
[9.9998116e-01 7.2626968e-13 1.8683095e-05 2.4710012e-11 1.1266836e-07]
Wrong sample 7
[0 0 1 0 0]
[9.5905000e-01 2.6465522e-04 3.2994002e-02 5.7674298e-04 7.1145785e-03]
Wrong sample 8
[1 0 0 0 0]
[0.2796548  0.5209429  0.01326527 0.18473299 0.00140402]
Wrong sample 9
[0 0 0 0 1]
[8.23830032e-06 1.97342550e-03 4.05455418e-02 8.33695829e-01
 1.23777024e-01]
Wrong sample 10
[0 0 0 0 1]
[5.5220958e-02 4.2097

In [ ]:
model.save_weights('./model/doc2vec_checkpoint')

## 2. GloVe + CNN

In [ ]:
X = text

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train

5187    us warships sail in disputed south china sea a...
622     vietnam seeking to fix labor shortage soon pm ...
2403    like prison singapore migrant workers suffer u...
3052    celebs show off neon colors model khanh linh d...
503     vietnam estimates covid loss at bln without th...
                              ...                        
2442    india to hit net zero climate target by modi b...
3886    like prison singapore migrant workers suffer u...
3371    coach park extends contract for another year p...
5148    two northern irishmen to be extradited over vi...
379     ancient town hoi an clears mud garbage after f...
Name: text, Length: 4233, dtype: object

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
words_to_index = tokenizer.word_index

In [ ]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map

In [ ]:
word_to_vec_map = read_glove_vector('/content/drive/Shareddrives/Intelligent System - Assignment /Glove/glove.6B.100d.txt')

In [ ]:
maxLen = 500
embed_vector_len = 100
vocab_len = len(words_to_index) + 1

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index, :] = embedding_vector

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = tf.keras.preprocessing.sequence.pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [ ]:
def glove_cnn(filters = 30, input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1D CNN
    input = tf.keras.Input(shape=(max_length,))
    embedding = tf.keras.layers.Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input)
    conv1 = tf.keras.layers.Conv1D(filters=filters, kernel_size=2, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(embedding)
    pool1 = tf.keras.layers.GlobalMaxPool1D()(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=filters, kernel_size=3, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(embedding)
    pool2 = tf.keras.layers.GlobalMaxPool1D()(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=filters, kernel_size=4, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(embedding)
    pool3 = tf.keras.layers.GlobalMaxPool1D()(conv3)
    concat = tf.keras.layers.concatenate([pool1, pool2, pool3], axis=1)
    drop1 = tf.keras.layers.Dropout(0.3)(concat)
    dense1 = tf.keras.layers.Dense(40, activation='relu')(drop1)
    drop2 = tf.keras.layers.Dropout(0.3)(dense1)
    outputs = tf.keras.layers.Dense(5, activation='softmax')(drop2)
    model = tf.keras.Model(inputs=input, outputs=outputs)
    
    # Compile
    model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model = glove_cnn(input_dim=vocab_len, output_dim=embed_vector_len, emb_matrix=emb_matrix, max_length=maxLen)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 100)     2655500     ['input_1[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 499, 30)      6030        ['embedding[0][0]']              
                                                                                                  
 conv1d_4 (Conv1D)              (None, 498, 30)      9030        ['embedding[0][0]']              
                                                                                              

In [ ]:
model.fit(X_train_indices, y_train, batch_size=256, epochs=100)

Epoch 1/100
17/17 [==============================] - 2s 39ms/step - loss: 1.6408 - accuracy: 0.3348
Epoch 2/100
17/17 [==============================] - 1s 30ms/step - loss: 1.4031 - accuracy: 0.4259
Epoch 3/100
17/17 [==============================] - 1s 31ms/step - loss: 1.2450 - accuracy: 0.5157
Epoch 4/100
17/17 [==============================] - 1s 30ms/step - loss: 1.0804 - accuracy: 0.5840
Epoch 5/100
17/17 [==============================] - 1s 30ms/step - loss: 0.9715 - accuracy: 0.6258
Epoch 6/100
17/17 [==============================] - 1s 30ms/step - loss: 0.8856 - accuracy: 0.6693
Epoch 7/100
17/17 [==============================] - 1s 31ms/step - loss: 0.8057 - accuracy: 0.6865
Epoch 8/100
17/17 [==============================] - 1s 30ms/step - loss: 0.7383 - accuracy: 0.7186
Epoch 9/100
17/17 [==============================] - 1s 30ms/step - loss: 0.6731 - accuracy: 0.7413
Epoch 10/100
17/17 [==============================] - 1s 30ms/step - loss: 0.6156 - accuracy: 0.7692

In [ ]:
X_test_indices = tokenizer.texts_to_sequences(X_test)
X_test_indices = tf.keras.preprocessing.sequence.pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
y_pred = model.predict(X_test_indices)
y_pred

array([[4.1099102e-03, 4.7654906e-04, 9.9507374e-01, 3.1725716e-04,
        2.2490971e-05],
       [7.0472894e-10, 3.7872205e-08, 4.6808363e-10, 1.0000000e+00,
        1.8691266e-11],
       [2.3075975e-04, 1.0102716e-03, 9.9661225e-01, 1.2715315e-03,
        8.7507715e-04],
       ...,
       [7.4665305e-09, 7.1847701e-08, 4.6903897e-10, 9.9999988e-01,
        1.5159884e-10],
       [5.2085606e-04, 9.9573416e-01, 1.1199829e-03, 2.5838215e-03,
        4.1143066e-05],
       [7.1688657e-05, 5.6396348e-05, 9.9986613e-01, 1.5884088e-06,
        4.1567855e-06]], dtype=float32)

In [ ]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.89      0.96      0.93       282
           1       0.87      0.76      0.81       121
           2       0.95      0.92      0.94       337
           3       0.95      0.99      0.97       219
           4       0.91      0.86      0.88       100

    accuracy                           0.92      1059
   macro avg       0.91      0.90      0.90      1059
weighted avg       0.92      0.92      0.92      1059



In [ ]:
# Analysis on wrong predictions
wrong_indices = np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1)
wrong_test = y_test[wrong_indices]
wrong_pred = y_pred[wrong_indices]
for i in range(min(wrong_test.shape[0], 100)):
  print('Wrong sample {}'.format(i))
  print(wrong_test[i])
  print(wrong_pred[i])

Wrong sample 0
[0 0 1 0 0]
[0.5310586  0.08281811 0.30003336 0.00290193 0.08318794]
Wrong sample 1
[0 0 1 0 0]
[9.999999e-01 9.054151e-09 9.169868e-08 2.545918e-09 3.574472e-10]
Wrong sample 2
[0 0 0 0 1]
[8.8328743e-05 8.3011844e-05 1.6493792e-05 9.9976069e-01 5.1449231e-05]
Wrong sample 3
[0 0 0 0 1]
[0.29207242 0.09664373 0.23951824 0.23701577 0.13474989]
Wrong sample 4
[0 0 1 0 0]
[0.9623685  0.00269282 0.02486289 0.0019141  0.00816178]
Wrong sample 5
[0 1 0 0 0]
[9.4822171e-06 4.0354580e-04 9.9958163e-01 4.4066060e-06 9.9295153e-07]
Wrong sample 6
[0 0 1 0 0]
[0.77514005 0.01623445 0.07097946 0.00255875 0.13508727]
Wrong sample 7
[1 0 0 0 0]
[0.17788915 0.05706117 0.18217169 0.00388235 0.57899565]
Wrong sample 8
[1 0 0 0 0]
[0.27880725 0.00509708 0.7138241  0.00121289 0.00105872]
Wrong sample 9
[0 1 0 0 0]
[9.9999464e-01 5.2093733e-06 6.2713106e-08 7.6562372e-08 2.1580833e-08]
Wrong sample 10
[0 1 0 0 0]
[2.0638910e-07 1.6275902e-07 1.0448159e-10 9.9999964e-01 2.6912730e-12]
Wrong

In [ ]:
model.evaluate(X_test_indices, y_test)

34/34 [==============================] - 0s 7ms/step - loss: 0.3531 - accuracy: 0.9216


[0.35307881236076355, 0.9216241836547852]